In [1]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
granularity = []

## Categories

In [3]:
pd.read_hdf('../generated/cat_rates.hdf5.csv.5', stop=10).head()

,category_id,platform,traffic_source,geo,cat_viewed
0,1000,1,1,-1,20.822754
1,1000,1,1,0,239.240352
2,1000,1,1,1,3.141090
3,1000,1,1,5,0.032317
4,1000,1,1,8,0.128752


In [4]:
rates = pd.read_hdf('../generated/cat_rates.hdf5.csv.5')
rates = pd.DataFrame({'weight':(rates.groupby(['category_id']+granularity)['cat_viewed'].sum() / sum(rates['cat_viewed']))}).reset_index()
rates['weight_sqrt'] = np.sqrt(rates['weight'])
rates.head()

,category_id,weight,weight_sqrt
0,1000,0.001395,0.037351
1,1100,0.017217,0.131212
2,1202,0.000468,0.021628
3,1203,0.004388,0.066241
4,1204,0.000935,0.030572


In [5]:
rates.count()

category_id    92
weight         92
weight_sqrt    92
dtype: int64

In [6]:
sum(rates['weight'])

0.9999999999997057

In [7]:
docs_cats = pd.read_csv("../download/documents_categories.csv")
docs_cats.head()

,document_id,category_id,confidence_level
0,1595802,1611,0.92
1,1595802,1610,0.07
2,1524246,1807,0.92
3,1524246,1608,0.07
4,1617787,1807,0.92


In [8]:
docs_cats = docs_cats.merge(rates, on='category_id', how='left').fillna(0)
docs_cats['value']=docs_cats['confidence_level']*docs_cats['weight_sqrt']
docs_cats['value2']=np.square(docs_cats['value'])
docs_cats.drop(['confidence_level','weight','weight_sqrt'], axis=1, inplace=True)
docs_cats.head()

,document_id,category_id,value,value2
0,1595802,1611,0.088385,0.007812
1,1595802,1610,0.007333,0.000054
2,1524246,1807,0.065103,0.004238
3,1524246,1608,0.006890,0.000047
4,1617787,1807,0.065103,0.004238


In [9]:
docs_norms2 = pd.DataFrame({'norm2':docs_cats.groupby(['document_id'] + granularity)['value2'].sum()}).reset_index()
docs_cats.drop(['value2'], axis=1, inplace=True)
docs_norms2.head()

,document_id,norm2
0,1,0.005663
1,2,0.009671
2,3,0.003675
3,4,0.010825
4,5,0.005242


### Train

In [10]:
page_addoc_dist = pd.read_csv("../generated/final/events_clicks_CTR_addocs_train.csv", usecols=['document_id', 'addoc_id'])
page_addoc_dist.count()

document_id    87141731
addoc_id       87141731
dtype: int64

In [11]:
page_addoc_dist = pd.DataFrame({'size':page_addoc_dist.groupby(['document_id', 'addoc_id']).size()}).reset_index().drop(['size'], axis=1)
page_addoc_dist.count()

document_id    28491064
addoc_id       28491064
dtype: int64

In [12]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

(756510, 164476)

In [13]:
page_addoc_diffs = page_addoc_dist.merge(docs_cats, left_on='addoc_id', right_on='document_id')
page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_diffs = page_addoc_diffs.merge(docs_cats, on=['document_id', 'category_id'])
page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
page_addoc_diffs.drop(['category_id', 'value_x', 'value_y'], axis=1, inplace=True)
page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id'])['value_2xy'].sum()}).reset_index()

page_addoc_diffs.head()

,document_id,addoc_id,value_2xy
0,5,1044364,0.010359
1,12,228823,0.005359
2,12,309413,0.027042
3,12,358632,0.000249
4,12,371327,0.012582


In [14]:
page_addoc_diffs.count()

document_id    9076297
addoc_id       9076297
value_2xy      9076297
dtype: int64

In [15]:
page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id'])
page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id'], right_on=['document_id'])
page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id'], how='left').fillna(0)
page_addoc_dist['dist_cats'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
page_addoc_dist.drop(['value_2xy'], axis=1, inplace=True)
page_addoc_dist['doc_norm_cats'] = np.sqrt( page_addoc_dist['norm2_x'] )
page_addoc_dist.drop(['norm2_x'], axis=1, inplace=True)
page_addoc_dist['ad_norm_cats'] = np.sqrt( page_addoc_dist['norm2_y'] )
page_addoc_dist.drop(['norm2_y'], axis=1, inplace=True)

page_addoc_dist.head()

,document_id,addoc_id,dist_cats,doc_norm_cats,ad_norm_cats
0,2,1489042,0.155770,0.098342,0.120802
1,452,1489042,0.135376,0.061104,0.120802
2,515,1489042,0.132236,0.053789,0.120802
3,1608,1489042,0.040956,0.080066,0.120802
4,3260,1489042,0.123571,0.026012,0.120802


In [16]:
page_addoc_dist.to_csv("../generated/final/page_addoc_cats_dist_w.csv", index=False)

In [17]:
page_addoc_dist.count()

document_id      27700275
addoc_id         27700275
dist_cats        27700275
doc_norm_cats    27700275
ad_norm_cats     27700275
dtype: int64

### Test

In [ ]:
page_addoc_dist = pd.read_csv("../generated/final/events_clicks_CTR_addocs_test.csv", usecols=['document_id', 'addoc_id'])
page_addoc_dist.count()

In [ ]:
page_addoc_dist = pd.DataFrame({'size':page_addoc_dist.groupby(['document_id', 'addoc_id']).size()}).reset_index().drop(['size'], axis=1)
page_addoc_dist.count()

In [ ]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

In [ ]:
page_addoc_diffs = page_addoc_dist.merge(docs_cats, left_on='addoc_id', right_on='document_id')
page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_diffs = page_addoc_diffs.merge(docs_cats, on=['document_id', 'category_id'])
page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
page_addoc_diffs.drop(['category_id', 'value_x', 'value_y'], axis=1, inplace=True)
page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id'])['value_2xy'].sum()}).reset_index()

page_addoc_diffs.head()

In [ ]:
page_addoc_diffs.count()

In [ ]:
page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id'])
page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id'], right_on=['document_id'])
page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id'], how='left').fillna(0)
page_addoc_dist['dist_cats'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
page_addoc_dist.drop(['norm2_x', 'norm2_y', 'value_2xy'], axis=1, inplace=True)

page_addoc_dist.head()

In [ ]:
page_addoc_dist.to_csv("../generated/final/page_addoc_cats_dist_test.csv", index=False)

In [ ]:
page_addoc_dist.count()